In [1]:
from keras.models import load_model
from array import array
from glob import glob
import pandas as pd
import collections
import uproot
import ROOT
import sys
from os import environ, path, mkdir, listdir
import multiprocessing
import subprocess
import h5py
environ['KMP_DUPLICATE_LIB_OK']='True'

def getGuess(df, index):
    try:
        prob_sig = df.loc[index, 'prob_sig']
    except:
        prob_sig = -999
    return prob_sig

def build_filelist(input_dir):
    filelist = collections.defaultdict(list)
    filelist['all'] = [fname for fname in glob('{}/*.csv'.format(input_dir))]
    return filelist

model = load_model('/localdata/Athar/CNN/models/{}.hdf5'
                   .format('model_7'))
all_data = pd.HDFStore('/localdata/Athar/CNN/datasets/feat_62.h5') 
if not path.isdir('/localdata/Athar/CNN/output_files'):
    mkdir('/localdata/Athar/CNN/output_files')

filelist = build_filelist('/localdata/Athar/CNN/13_particle_features_3')
for _, files in filelist.items():
    for ifile in files:
        print ('Processing file: {}'.format(ifile))
        fname = ifile
        MakeCsvDataFrame = ROOT.RDF.MakeCsvDataFrame
        csv_file = MakeCsvDataFrame(ifile)
        input_dict = csv_file.AsNumpy()
        output_file = pd.DataFrame.from_dict(input_dict)
        data = all_data['nominal']
        print('Sample' + fname)
        sample = data[(data['sample_names'] == fname)]
        to_classify = sample[['m_B1', 'pt_B1', 'm_B2', 'pt_B2', 'm_J1', 'pt_J1', 'm_J2', 'pt_J2',
              'm_L1', 'pt_L1', 'm_N1', 'pt_N1', 'm_H', 'pt_H', 'm_t11', 'pt_t11', 'm_t12', 
              'pt_t12', 'm_t21', 'pt_t21', 'm_t22', 'pt_t22', 'm_W1', 'pt_W1', 'm_W2', 
              'pt_W2', 'dr_B1_B2', 'dr_B1_J1', 'dr_B1_J2', 'dr_B1_L1', 
              'dr_B1_N1', 'dr_B1_H', 'dr_B1_t11', 'dr_B1_t12', 'dr_B1_t21', 'dr_B1_t22', 
              'dr_B1_W1', 'dr_B1_W2', 'dr_B2_B1', 'dr_B2_J1', 'dr_B2_J2', 'dr_B2_L1', 'dr_B2_N1', 
              'dr_B2_H', 'dr_B2_t11', 'dr_B2_t12', 'dr_B2_t21', 'dr_B2_t22', 'dr_B2_W1', 
              'dr_B2_W2', 'dr_J1_B1', 'dr_J1_B2', 'dr_J1_J2', 'dr_J1_L1', 'dr_J1_N1', 'dr_J1_H', 
              'dr_J1_t11', 'dr_J1_t12', 'dr_J1_t21', 'dr_J1_t22', 'dr_J1_W1', 'dr_J1_W2', 'dr_J2_B1', 
              'dr_J2_B2', 'dr_J2_J1', 'dr_J2_L1', 'dr_J2_N1', 'dr_J2_H', 'dr_J2_t11', 'dr_J2_t12', 
              'dr_J2_t21', 'dr_J2_t22', 'dr_J2_W1', 'dr_J2_W2', 'dr_L1_B1', 'dr_L1_B2', 'dr_L1_J1', 
              'dr_L1_J2', 'dr_L1_N1', 'dr_L1_H', 'dr_L1_t11', 'dr_L1_t12', 'dr_L1_t21', 'dr_L1_t22', 
              'dr_L1_W1', 'dr_L1_W2', 'dr_N1_B1', 'dr_N1_B2', 'dr_N1_J1', 'dr_N1_J2', 'dr_N1_L1', 
              'dr_N1_H', 'dr_N1_t11', 'dr_N1_t12', 'dr_N1_t21', 'dr_N1_t22', 'dr_N1_W1', 'dr_N1_W2', 
              'dr_H_B1', 'dr_H_B2', 'dr_H_J1', 'dr_H_J2', 'dr_H_L1', 'dr_H_N1', 'dr_H_t11', 'dr_H_t12', 
              'dr_H_t21', 'dr_H_t22', 'dr_H_W1', 'dr_H_W2', 'dr_t11_B1', 'dr_t11_B2', 'dr_t11_J1', 
              'dr_t11_J2', 'dr_t11_L1', 'dr_t11_N1', 'dr_t11_H', 'dr_t11_t12', 'dr_t11_t21', 'dr_t11_t22', 
              'dr_t11_W1', 'dr_t11_W2', 'dr_t12_B1', 'dr_t12_B2', 'dr_t12_J1', 'dr_t12_J2', 'dr_t12_L1', 
              'dr_t12_N1', 'dr_t12_H', 'dr_t12_t11', 'dr_t12_t21', 'dr_t12_t22', 'dr_t12_W1', 'dr_t12_W2', 
              'dr_t21_B1', 'dr_t21_B2', 'dr_t21_J1', 'dr_t21_J2', 'dr_t21_L1', 'dr_t21_N1', 'dr_t21_H', 
              'dr_t21_t11', 'dr_t21_t12', 'dr_t21_t22', 'dr_t21_W1', 'dr_t21_W2', 'dr_t22_B1', 'dr_t22_B2', 
              'dr_t22_J1', 'dr_t22_J2', 'dr_t22_L1', 'dr_t22_N1', 'dr_t22_H', 'dr_t22_t11', 'dr_t22_t12', 
              'dr_t22_t21', 'dr_t22_W1', 'dr_t22_W2', 'dr_W1_B1', 'dr_W1_B2', 'dr_W1_J1', 'dr_W1_J2', 
              'dr_W1_L1', 'dr_W1_N1', 'dr_W1_H', 'dr_W1_t11', 'dr_W1_t12', 'dr_W1_t21', 'dr_W1_t22', 
              'dr_W1_W2', 'dr_W2_B1', 'dr_W2_B2', 'dr_W2_J1', 'dr_W2_J2', 'dr_W2_L1', 'dr_W2_N1', 'dr_W2_H', 
              'dr_W2_t11', 'dr_W2_t12', 'dr_W2_t21', 'dr_W2_t22', 'dr_W2_W1', 'Hness', 'Tness', 
              'd_eta_B1_B2', 'd_phi_B1_B2', 'd_eta_B1_J1', 'd_phi_B1_J1', 'd_eta_B1_J2', 'd_phi_B1_J2', 
              'd_eta_B1_L1', 'd_phi_B1_L1', 'd_eta_B1_N1', 'd_phi_B1_N1', 'd_eta_B1_H', 'd_phi_B1_H', 
              'd_eta_B1_t11', 'd_phi_B1_t11', 'd_eta_B1_t12', 'd_phi_B1_t12', 'd_eta_B1_t21', 'd_phi_B1_t21', 
              'd_eta_B1_t22', 'd_phi_B1_t22', 'd_eta_B1_W1', 'd_phi_B1_W1', 'd_eta_B1_W2', 'd_phi_B1_W2', 
              'd_eta_B2_B1', 'd_phi_B2_B1', 'd_eta_B2_J1', 'd_phi_B2_J1', 'd_eta_B2_J2', 'd_phi_B2_J2', 
              'd_eta_B2_L1', 'd_phi_B2_L1', 'd_eta_B2_N1', 'd_phi_B2_N1', 'd_eta_B2_H', 'd_phi_B2_H', 
              'd_eta_B2_t11', 'd_phi_B2_t11', 'd_eta_B2_t12', 'd_phi_B2_t12', 'd_eta_B2_t21', 'd_phi_B2_t21', 
              'd_eta_B2_t22', 'd_phi_B2_t22', 'd_eta_B2_W1', 'd_phi_B2_W1', 'd_eta_B2_W2', 'd_phi_B2_W2', 
              'd_eta_J1_B1', 'd_phi_J1_B1', 'd_eta_J1_B2', 'd_phi_J1_B2', 'd_eta_J1_J2', 'd_phi_J1_J2', 
              'd_eta_J1_L1', 'd_phi_J1_L1', 'd_eta_J1_N1', 'd_phi_J1_N1', 'd_eta_J1_H', 'd_phi_J1_H', 
              'd_eta_J1_t11', 'd_phi_J1_t11', 'd_eta_J1_t12', 'd_phi_J1_t12', 'd_eta_J1_t21', 'd_phi_J1_t21', 
              'd_eta_J1_t22', 'd_phi_J1_t22', 'd_eta_J1_W1', 'd_phi_J1_W1', 'd_eta_J1_W2', 'd_phi_J1_W2', 
              'd_eta_J2_B1', 'd_phi_J2_B1', 'd_eta_J2_B2', 'd_phi_J2_B2', 'd_eta_J2_J1', 'd_phi_J2_J1', 
              'd_eta_J2_L1', 'd_phi_J2_L1', 'd_eta_J2_N1', 'd_phi_J2_N1', 'd_eta_J2_H', 'd_phi_J2_H', 
              'd_eta_J2_t11', 'd_phi_J2_t11', 'd_eta_J2_t12', 'd_phi_J2_t12', 'd_eta_J2_t21', 'd_phi_J2_t21', 
              'd_eta_J2_t22', 'd_phi_J2_t22', 'd_eta_J2_W1', 'd_phi_J2_W1', 'd_eta_J2_W2', 'd_phi_J2_W2', 
              'd_eta_L1_B1', 'd_phi_L1_B1', 'd_eta_L1_B2', 'd_phi_L1_B2', 'd_eta_L1_J1', 'd_phi_L1_J1', 
              'd_eta_L1_J2', 'd_phi_L1_J2', 'd_eta_L1_N1', 'd_phi_L1_N1', 'd_eta_L1_H', 'd_phi_L1_H', 
              'd_eta_L1_t11', 'd_phi_L1_t11', 'd_eta_L1_t12', 'd_phi_L1_t12', 'd_eta_L1_t21', 'd_phi_L1_t21', 
              'd_eta_L1_t22', 'd_phi_L1_t22', 'd_eta_L1_W1', 'd_phi_L1_W1', 'd_eta_L1_W2', 'd_phi_L1_W2', 
              'd_eta_N1_B1', 'd_phi_N1_B1', 'd_eta_N1_B2', 'd_phi_N1_B2', 'd_eta_N1_J1', 'd_phi_N1_J1', 
              'd_eta_N1_J2', 'd_phi_N1_J2', 'd_eta_N1_L1', 'd_phi_N1_L1', 'd_eta_N1_H', 'd_phi_N1_H', 
              'd_eta_N1_t11', 'd_phi_N1_t11', 'd_eta_N1_t12', 'd_phi_N1_t12', 'd_eta_N1_t21', 'd_phi_N1_t21', 
              'd_eta_N1_t22', 'd_phi_N1_t22', 'd_eta_N1_W1', 'd_phi_N1_W1', 'd_eta_N1_W2', 'd_phi_N1_W2', 
              'd_eta_H_B1', 'd_phi_H_B1', 'd_eta_H_B2', 'd_phi_H_B2', 'd_eta_H_J1', 'd_phi_H_J1', 
              'd_eta_H_J2', 'd_phi_H_J2', 'd_eta_H_L1', 'd_phi_H_L1', 'd_eta_H_N1', 'd_phi_H_N1', 
              'd_eta_H_t11', 'd_phi_H_t11', 'd_eta_H_t12', 'd_phi_H_t12', 'd_eta_H_t21', 'd_phi_H_t21', 
              'd_eta_H_t22', 'd_phi_H_t22', 'd_eta_H_W1', 'd_phi_H_W1', 'd_eta_H_W2', 'd_phi_H_W2', 
              'd_eta_t11_B1', 'd_phi_t11_B1', 'd_eta_t11_B2', 'd_phi_t11_B2', 'd_eta_t11_J1', 'd_phi_t11_J1', 
              'd_eta_t11_J2', 'd_phi_t11_J2', 'd_eta_t11_L1', 'd_phi_t11_L1', 'd_eta_t11_N1', 'd_phi_t11_N1', 
              'd_eta_t11_H', 'd_phi_t11_H', 'd_eta_t11_t12', 'd_phi_t11_t12', 'd_eta_t11_t21', 
              'd_phi_t11_t21', 'd_eta_t11_t22', 'd_phi_t11_t22', 'd_eta_t11_W1', 'd_phi_t11_W1', 
              'd_eta_t11_W2', 'd_phi_t11_W2', 'd_eta_t12_B1', 'd_phi_t12_B1', 'd_eta_t12_B2', 
              'd_phi_t12_B2', 'd_eta_t12_J1', 'd_phi_t12_J1', 'd_eta_t12_J2', 'd_phi_t12_J2', 
              'd_eta_t12_L1', 'd_phi_t12_L1', 'd_eta_t12_N1', 'd_phi_t12_N1', 'd_eta_t12_H', 
              'd_phi_t12_H', 'd_eta_t12_t11', 'd_phi_t12_t11', 'd_eta_t12_t21', 'd_phi_t12_t21', 
              'd_eta_t12_t22', 'd_phi_t12_t22', 'd_eta_t12_W1', 'd_phi_t12_W1', 'd_eta_t12_W2', 
              'd_phi_t12_W2', 'd_eta_t21_B1', 'd_phi_t21_B1', 'd_eta_t21_B2', 'd_phi_t21_B2', 
              'd_eta_t21_J1', 'd_phi_t21_J1', 'd_eta_t21_J2', 'd_phi_t21_J2', 'd_eta_t21_L1', 
              'd_phi_t21_L1', 'd_eta_t21_N1', 'd_phi_t21_N1', 'd_eta_t21_H', 'd_phi_t21_H', 
              'd_eta_t21_t11', 'd_phi_t21_t11', 'd_eta_t21_t12', 'd_phi_t21_t12', 'd_eta_t21_t22', 
              'd_phi_t21_t22', 'd_eta_t21_W1', 'd_phi_t21_W1', 'd_eta_t21_W2', 'd_phi_t21_W2', 
              'd_eta_t22_B1', 'd_phi_t22_B1', 'd_eta_t22_B2', 'd_phi_t22_B2', 'd_eta_t22_J1', 
              'd_phi_t22_J1', 'd_eta_t22_J2', 'd_phi_t22_J2', 'd_eta_t22_L1', 'd_phi_t22_L1', 
              'd_eta_t22_N1', 'd_phi_t22_N1', 'd_eta_t22_H', 'd_phi_t22_H', 'd_eta_t22_t11', 
              'd_phi_t22_t11', 'd_eta_t22_t12', 'd_phi_t22_t12', 'd_eta_t22_t21', 'd_phi_t22_t21', 
              'd_eta_t22_W1', 'd_phi_t22_W1', 'd_eta_t22_W2', 'd_phi_t22_W2', 'd_eta_W1_B1', 
              'd_phi_W1_B1', 'd_eta_W1_B2', 'd_phi_W1_B2', 'd_eta_W1_J1', 'd_phi_W1_J1', 
              'd_eta_W1_J2', 'd_phi_W1_J2', 'd_eta_W1_L1', 'd_phi_W1_L1', 'd_eta_W1_N1', 
              'd_phi_W1_N1', 'd_eta_W1_H', 'd_phi_W1_H', 'd_eta_W1_t11', 'd_phi_W1_t11', 
              'd_eta_W1_t12', 'd_phi_W1_t12', 'd_eta_W1_t21', 'd_phi_W1_t21', 'd_eta_W1_t22', 
              'd_phi_W1_t22', 'd_eta_W1_W2', 'd_phi_W1_W2', 'd_eta_W2_B1', 'd_phi_W2_B1', 
              'd_eta_W2_B2', 'd_phi_W2_B2', 'd_eta_W2_J1', 'd_phi_W2_J1', 'd_eta_W2_J2', 
              'd_phi_W2_J2', 'd_eta_W2_L1', 'd_phi_W2_L1', 'd_eta_W2_N1', 'd_phi_W2_N1', 
              'd_eta_W2_H', 'd_phi_W2_H', 'd_eta_W2_t11', 'd_phi_W2_t11', 'd_eta_W2_t12', 
              'd_phi_W2_t12', 'd_eta_W2_t21', 'd_phi_W2_t21', 'd_eta_W2_t22', 'd_phi_W2_t22', 
              'd_eta_W2_W1', 'd_phi_W2_W1'
             ]]
        
        print(to_classify.shape)
        guesses = model.predict(to_classify.values, verbose=True)
        out = data.copy()
        output_file['prob_sig'] = guesses[:,0]
        print(output_file)
        out_name = ifile.split('/')[-1].replace('.csv', '_feature_7.csv')
        output_file.to_csv('/localdata/Athar/CNN/output_files/{}'.format(out_name), index=False)

Welcome to JupyROOT 6.24/06
Processing file: /localdata/Athar/CNN/13_particle_features_3/tt.csv
Sample/localdata/Athar/CNN/13_particle_features_3/tt.csv
(244917, 496)
7654/7654 [==============================] - 4s 537us/step
             eB1       pxB1      pyB1       pzB1       eB2       pxB2  \
0       107.1910   92.07420   34.1533  -40.45480   51.7417   -7.99619   
1       108.4730  -81.63750   59.8673  -37.60060  172.9410   29.55680   
2       120.7580  -76.16870   19.6312   90.96610   60.3143   53.01330   
3       103.1690   94.29710  -40.7002   -3.58747   70.2221   35.04040   
4       591.4550   60.11420 -304.8610 -499.81600  228.9360 -146.00900   
...          ...        ...       ...        ...       ...        ...   
244912   82.6719    4.96326  -80.9572    7.76526   49.4617  -19.85860   
244913  149.2640   57.54540   32.2669  133.70100  130.7430  -40.51460   
244914  167.4160 -130.86000  -88.0257   53.81650  104.0230  -16.44490   
244915   59.8195   41.75880   12.0000  -40.1